In [1]:
from ms4alg_pipelines import synthesize_sample_dataset
from ms4alg_pipelines import sort_dataset
import os

In [2]:
synthesize_sample_dataset(dataset_dir='dataset',opts={'verbose':'minimal'})

RUNNING: ml-run-process ephys.synthesize_random_waveforms --inputs --parameters M:4 average_peak_amplitude:100 upsamplefac:13 --outputs geometry_out:dataset/geom.csv waveforms_out:dataset/waveforms_true.mda --verbose=minimal
RUNNING: ml-run-process ephys.synthesize_random_firings --inputs --parameters duration:600 --outputs firings_out:dataset/firings_true.mda --verbose=minimal
RUNNING: ml-run-process ephys.synthesize_timeseries --inputs firings:dataset/firings_true.mda waveforms:dataset/waveforms_true.mda --parameters duration:600 noise_level:10 waveform_upsamplefac:13 --outputs timeseries_out:dataset/raw.mda.prv --verbose=minimal


In [3]:
sort_dataset(dataset_dir='dataset',output_dir='output',adjacency_radius=-1,detect_threshold=3,opts={'verbose':'minimal'})

RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:dataset/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:30000 --outputs timeseries_out:output/filt.mda.prv --verbose=minimal
RUNNING: ml-run-process ephys.whiten --inputs timeseries:output/filt.mda.prv --parameters --outputs timeseries_out:output/pre.mda.prv --verbose=minimal
RUNNING: ml-run-process ms4alg.sort --inputs geom:dataset/geom.csv timeseries:output/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:1 detect_threshold:3 --outputs firings_out:output/firings.mda --verbose=minimal
RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:output/firings.mda timeseries:output/pre.mda.prv --parameters samplerate:30000 --outputs cluster_metrics_out:/tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_cluster_metrics_out_c2fdbc02c1895ed5197f0b337278fb363092df59.prv --verbose=minimal
RUNNING: ml-run-process ms3.isolation_metrics --inputs firings:output/firings.mda timeseries:output/pre.mda.prv --pa

In [6]:
# Run this only if you are running jupyter lab locally and have ephys-viz installed:
os.system('ev-timeseries {} --firings={}'.format('output/filt.mda.prv','output/firings_curated.mda'))

0